In [1]:
import random
import time
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

# Simulation of the stock market
def simulate_stock_market(stock_symbol, duration=60, interval=5):
    print(f"Simulating {stock_symbol} stock market...")
    prices = []
    ohlc_data = []

    for _ in range(duration // interval):
        open_price = random.uniform(50, 150)
        close_price = random.uniform(50, 150)
        high_price = max(open_price, close_price) + random.uniform(0, 5)
        low_price = min(open_price, close_price) - random.uniform(0, 5)

        prices.append(close_price)
        current_time = time.strftime('%H:%M:%S')

        ohlc_data.append({
            "Time": current_time,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Close": close_price
        })

        print(f"{current_time} - {stock_symbol}: ${close_price:.2f}")
        time.sleep(interval)

    return ohlc_data, prices[-1]

# The final price of the stock
def generate_stock_prices(final_prices):
    return [{"name": stock_name, "price": price} for stock_name, price in final_prices.items()]

# Merge Sort
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        left = arr[:mid]
        right = arr[mid:]
        merge_sort(left)
        merge_sort(right)

        i = j = k = 0
        while i < len(left) and j < len(right):
            if left[i]["price"] < right[j]["price"]:
                arr[k] = left[i]
                i += 1
            else:
                arr[k] = right[j]
                j += 1
            k += 1

        while i < len(left):
            arr[k] = left[i]
            i += 1
            k += 1
        while j < len(right):
            arr[k] = right[j]
            j += 1
            k += 1

#  Plotting the Graph of the stocks
def plot_stock_prices(stocks):
    names = [stock["name"] for stock in stocks]
    prices = [stock["price"] for stock in stocks]
    plt.bar(names, prices, color="blue")
    plt.xlabel("Stocks")
    plt.ylabel("Prices")
    plt.title("Stock Prices")
    plt.show()

# Candlestick Chart
def show_candlestick_chart(ohlc_data, stock_symbol):
    df = pd.DataFrame(ohlc_data)

    fig = go.Figure(data=[go.Candlestick(
        x=df['Time'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        increasing_line_color='green',
        decreasing_line_color='red'
    )])

    fig.update_layout(
        title=f"{stock_symbol} Stock Candlestick Chart",
        xaxis_title="Time",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=False,
        template="plotly_dark"
    )
    fig.show()

# User input to invest in stock
def invest_in_stock(stocks, user_portfolio, ohlc_data):
    print("\nAvailable Stocks:")
    for i, stock in enumerate(stocks):
        print(f"{i+1}. {stock['name']} - ${stock['price']:.2f}")
    try:
        stock_choice = int(input("Choose a stock to invest in: ")) - 1
        if 0 <= stock_choice < len(stocks):
            stock = stocks[stock_choice]
            investment_amount = float(input(f"Enter the amount to invest in {stock['name']}: $"))
            user_portfolio[stock['name']] = {
                "stock_name": stock['name'],
                "buy_price": stock['price'],
                "investment_amount": investment_amount,
                "ohlc_data": ohlc_data[stock['name']]
            }
            print(f"\nYou invested ${investment_amount:.2f} in {stock['name']} at ${stock['price']:.2f}.")
        else:
            print("Invalid stock choice!")
    except ValueError:
        print("Invalid input! Please enter a valid number.")

# User input to veiw portfolio
def view_portfolio(user_portfolio):
    print("\nYour Portfolio:")
    if user_portfolio:
        for investment in user_portfolio.values():
            print(f"Stock: {investment['stock_name']}, Invested: ${investment['investment_amount']:.2f}, Buy Price: ${investment['buy_price']:.2f}")
    else:
        print("You have no investments yet!")

# User input to sell stock
def sell_stock(user_portfolio, stocks):
    print("\nYour Portfolio:")
    if not user_portfolio:
        print("You have no stocks to sell!")
        return

    for i, stock in enumerate(user_portfolio.keys()):
        print(f"{i+1}. {stock}")

    try:
        stock_choice = int(input("Choose a stock to sell: ")) - 1
        stock_name = list(user_portfolio.keys())[stock_choice]
        sell_price = next(stock['price'] for stock in stocks if stock['name'] == stock_name)
        investment = user_portfolio.pop(stock_name)

        profit_loss = sell_price - investment['buy_price']
        print(f"\nYou sold {stock_name} at ${sell_price:.2f}.")
        if profit_loss > 0:
            print(f"Profit: ${profit_loss * investment['investment_amount'] / investment['buy_price']:.2f}")
        else:
            print(f"Loss: ${abs(profit_loss * investment['investment_amount'] / investment['buy_price']):.2f}")
    except (ValueError, IndexError, StopIteration):
        print("Invalid choice! Please select a valid stock.")

# Main Menu of the stock market
def menu(final_prices, ohlc_data):
    stocks = generate_stock_prices(final_prices)
    user_portfolio = {}

    print("Welcome to the OKAE Stock Market!")
    while True:
        print("\n1. View Stocks")
        print("2. Invest in a Stock")
        print("3. View Portfolio")
        print("4. Sort Stocks by Price")
        print("5. Show Candlestick Chart")
        print("6. Sell a Stock")
        print("7. Exit")
        choice = input("Enter your choice: ")
        if choice == "1":
            print("\nAvailable Stocks:")
            for stock in stocks:
                print(f"{stock['name']}: ${stock['price']:.2f}")
        elif choice == "2":
            invest_in_stock(stocks, user_portfolio, ohlc_data)
        elif choice == "3":
            view_portfolio(user_portfolio)
        elif choice == "4":
            merge_sort(stocks)
            print("\nStocks sorted by price:")
            for stock in stocks:
                print(f"{stock['name']}: ${stock['price']:.2f}")
        elif choice == "5":
            if user_portfolio:
                print("\nYour Investments:")
                for i, stock in enumerate(user_portfolio.keys()):
                    print(f"{i+1}. {stock}")
                try:
                    stock_choice = int(input("Choose a stock to view its candlestick chart: ")) - 1
                    stock_name = list(user_portfolio.keys())[stock_choice]
                    show_candlestick_chart(user_portfolio[stock_name]['ohlc_data'], stock_name)
                except (ValueError, IndexError):
                    print("Invalid choice! Please select a valid stock.")
            else:
                print("You need to invest in a stock before viewing the candlestick chart.")
        elif choice == "6":
            sell_stock(user_portfolio, stocks)
        elif choice == "7":
            print("Exiting...")
            break
        else:
            print("Invalid choice! Please try again.")

# Stocks
if __name__ == "__main__":
    stock_symbols = ["Apple", "Google", "Amazon", "Microsoft", "Tesla"]
    final_prices = {}
    ohlc_data = {}
    for stock in stock_symbols:
        ohlc, final_price = simulate_stock_market(stock, duration=10, interval=2)
        ohlc_data[stock] = ohlc
        final_prices[stock] = round(final_price, 2)
    menu(final_prices, ohlc_data)

Simulating Apple stock market...
13:12:28 - Apple: $133.83
13:12:30 - Apple: $144.43
13:12:32 - Apple: $64.94
13:12:34 - Apple: $92.05
13:12:36 - Apple: $75.33
Simulating Google stock market...
13:12:38 - Google: $127.67
13:12:40 - Google: $148.98
13:12:42 - Google: $132.95
13:12:44 - Google: $107.46
13:12:46 - Google: $94.88
Simulating Amazon stock market...
13:12:48 - Amazon: $83.30
13:12:50 - Amazon: $69.60
13:12:52 - Amazon: $103.11
13:12:54 - Amazon: $99.09
13:12:56 - Amazon: $52.08
Simulating Microsoft stock market...
13:12:58 - Microsoft: $92.23
13:13:00 - Microsoft: $103.70
13:13:02 - Microsoft: $52.86
13:13:04 - Microsoft: $137.26
13:13:06 - Microsoft: $95.69
Simulating Tesla stock market...
13:13:08 - Tesla: $127.38
13:13:10 - Tesla: $60.86
13:13:12 - Tesla: $125.32
13:13:14 - Tesla: $116.44
13:13:16 - Tesla: $117.44
Welcome to the OKAE Stock Market!

1. View Stocks
2. Invest in a Stock
3. View Portfolio
4. Sort Stocks by Price
5. Show Candlestick Chart
6. Sell a Stock
7. Exi


1. View Stocks
2. Invest in a Stock
3. View Portfolio
4. Sort Stocks by Price
5. Show Candlestick Chart
6. Sell a Stock
7. Exit
Enter your choice: 6

Your Portfolio:
1. Apple
Choose a stock to sell: 1

You sold Apple at $75.33.
Loss: $0.00

1. View Stocks
2. Invest in a Stock
3. View Portfolio
4. Sort Stocks by Price
5. Show Candlestick Chart
6. Sell a Stock
7. Exit
Enter your choice: 7
Exiting...
